# Accessing ACME
### Ray Smith

In [102]:
# Selenium Imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

# Data Handling Imports
import numpy as np
import pandas as pd

# Datetime Imports
import datetime

In [2]:
# Settings
url = "https://acme.wisc.edu/tools/schedule/schedule.php"

login_secret = "login.secret"
duo_secret = "duo.secret"

options = Options()
# options.headless = True

In [3]:
# Get Secrets
with open(login_secret) as f:
    user = f.readline()[:-1]
    password = f.readline()

In [4]:
# Date conversion functions
# String form = date=YYYY-MM-DD

# URL String to Date
def string_to_date(string):
    parts = string.split('-')
    year = int(parts[0])
    month = int(parts[1])
    day = int(parts[2])
    date = datetime.date(year=year, month=month, day=day)
    return date

# Date to URL String
def date_to_string(date):
    year = date.year
    month = date.month
    day = date.day
    return str(year) + '-' + str(month) + '-' + str(day)

In [5]:
# Login to ACME
def acme_login():
    browser = webdriver.Chrome(options=options)
    browser.get(url)

    # Fill in username
    user_field = browser.find_element_by_id('j_username')
    user_field.send_keys(user)

    # Fill in password
    pass_field = browser.find_element_by_id('j_password')
    pass_field.send_keys(password)

    # Click login
    login_button = browser.find_element_by_name('_eventId_proceed')
    login_button.click()
    
    # Return ACME pointed browser
    return browser

In [126]:
# Get Schedule Table by Date (as string)
def get_schedule_by_date(browser, date):
    # Access page for specified date
    new_url = url + '?date=' + date
    browser.get(new_url)
    
    # Get table headers
    header_html = browser.find_element_by_xpath("//thead[1]/tr").get_attribute('innerHTML')
    headers = []
    header_text = header_html.split(" ")
    for line in header_text:
        if line.startswith('id='):
            parts = line.split('"')
            header = parts[1]
            parts = header.split('_')
            headers.append(parts[1])
    headers.append("total")
    
    # Get table
    table = browser.find_element_by_id('sch_table_verticle')
    table_html = table.get_attribute('outerHTML')
    df = pd.read_html(table_html)[0]
    df.drop(df.tail(2).index,inplace=True)
    
    # Set headers of dataframe
    columns = df.columns
    new_names = {}
    for i in range(len(columns)):
        new_names[columns[i]] = headers[i]
    df = df.rename(columns=new_names)
    df = df.set_index('time')
    df = df.drop(columns=['total'])
    
    # Split up agents
    for col in df:
        data = df[col]
        new_data = []
        for row in data:
            if type(row) != str:
                new_data.append(None)
            else:
                agents = [row[i:i+4] for i in range(0, len(row), 4)]
                new_data.append(agents)
        df[col] = new_data
    
    return df

In [62]:
# Login
browser = acme_login()

In [129]:
# Access certain date
date = '2020-12-15'
table= get_schedule_by_date(browser, date)
table

,hdqaremote,phonesfte,phonesremote,chatemailremote,hdl1project,walkinsd,repair,wihdonsitefte,wihdappt,stloutreach,meeting
time,,,,,,,,,,,
7:00AM,[KELL],None,"[RWER, MJMO]",[MBER],None,None,None,None,None,None,None
7:30AM,[KELL],None,"[RWER, MJMO]",[MBER],None,None,None,None,None,None,None
8:00AM,[KELL],None,"[RWER, GARC, MJMO, GOFF]",[MBER],None,None,None,None,None,None,None
8:30AM,[KELL],None,"[RWER, GARC, MJMO, GOFF]",[MBER],None,None,None,[YANG],None,None,None
9:00AM,[KELL],None,"[RWER, RMAJ, GARC, ATRI, MJMO, GOFF]",[MBER],[JADE],[ADWI],[ALZA],"[HPRI, YANG]",None,None,None
9:30AM,[KELL],None,"[RWER, RMAJ, GARC, ATRI, MJMO, GOFF]",[MBER],[JADE],[ADWI],[ALZA],"[HPRI, YANG]",None,None,None
10:00AM,[KELL],None,"[RWER, VRNA, SPEK, RMAJ, ATRI, GOFF]",[MBER],"[CACH, JADE]",[RAYS],[ALZA],"[HPRI, YANG]",[ADWI],None,None
10:30AM,[KELL],None,"[RWER, VRNA, ELLI, SPEK, RMAJ, ATRI, GOFF]",[MBER],"[CACH, JADE]","[RAYS, ADWI]",[ALZA],"[HPRI, YANG]",None,None,None
11:00AM,[RMAJ],[JADE],"[KELL, MBER, ELLI, SPEK, GUZM, MNEL, GOFF]",[VRNA],[CACH],[ADWI],[ALZA],"[HPRI, YANG]",[RAYS],"[RWER, DUNN]",None


In [130]:
# Close browser
browser.close()